In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler   # u otros scalers
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, r2_score
from distancia import haversine

In [ ]:
### carga datos de dataset en dataframe
file_path= 'uber_fares.csv'

df = pd.read_csv(file_path)
### visualizacion de algunos datos
df.head()

#### Contexto  
El proyecto trata sobre **Uber Inc.**, la compañía de taxis más grande del mundo. En este trabajo, nuestro objetivo es **predecir la tarifa de futuros viajes**.  

Uber brinda servicio a millones de clientes cada día, por lo que gestionar adecuadamente sus datos es clave para desarrollar nuevas estrategias de negocio y obtener mejores resultados.  

### Variables del conjunto de datos  

**Variables explicativas:**  
- **key**: identificador único de cada viaje.  
- **pickup_datetime**: fecha y hora en que se inició el viaje.  
- **passenger_count**: cantidad de pasajeros en el vehículo (dato ingresado por el conductor).  
- **pickup_longitude**: longitud del punto de inicio del viaje.  
- **pickup_latitude**: latitud del punto de inicio del viaje.  
- **dropoff_longitude**: longitud del punto de destino.  
- **dropoff_latitude**: latitud del punto de destino.  

**Variable objetivo (target):**  
- **fare_amount**: costo del viaje en dólares.  

In [ ]:
### Columnas, ¿cuáles son variables numéricas y cuales variables categóricas?
df.columns

En este dataset trabajaremos con algunas columnas de interes, las cuales clasificamos a continuacion dependiendo el tipo de variable:

*   **fare_amount:**   cuantitativa continua
*   **pickup_datetime:** cualitativa nominal
*   **pickup_longitude:** cuantitativa continua
*   **pickup_latitude:** cuantitativa continua
*   **dropoff_longitude:** cuantitativa continua
*   **dropoff_latitude:** cuantitativa continua
*   **passenger_count:** cuantitativa discreta

Lo primero que realizaremos será utilizar el método `.info()` para verificar que el tipo de dato en cada variable es correcto, detectar la presencia de valores nulos y valores atípicos.

In [ ]:
# Descartar valores del target negativos, cero o nulos
# Mostrar la cantidad de registros antes y después de la limpieza
print(f"Cantidad de registros antes de la limpieza: {len(df)}") 
df = df[df['fare_amount'] > 0]
print(f"Cantidad de registros después de la limpieza: {len(df)}")
df = df.dropna(subset=['fare_amount'])
print(f"Cantidad de registros después de eliminar nulos: {len(df)}")    

### Agregar las variables de distancia y tiempo antes de dividir en train y test

In [ ]:
# Cambia el tipo de dato object -> datetime
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])                   
# Elimina las filas con valores nulos
#train_df.dropna(inplace=True)     #No haria falta lo hacemos con imputacion

# Eliminación de columnas que no son de interes
df = df.drop('key', axis = 1)
df = df.drop('date', axis = 1)

In [ ]:
# Agregar mas caraterísticas a partir de las coordenadas geográficas
df["delta_lat"] = df["dropoff_latitude"] - df["pickup_latitude"]
df["delta_lon"] = df["dropoff_longitude"] - df["pickup_longitude"]

df["distance_km"] = df.apply(
    lambda row: haversine(
        row["pickup_longitude"], row["pickup_latitude"],
        row["dropoff_longitude"], row["dropoff_latitude"]
    ),
    axis=1
)
df.head()

In [ ]:
# obtener el dia de la semana y la hora del dia a partir de pickup_datetime
df['pickup_day_of_week'] = df['pickup_datetime'].dt.dayofweek
df['pickup_hour'] = df['pickup_datetime'].dt.hour
# agregar campo franja horaria a partir del campo pickup_hour
def franja_horaria(hora):
    if 0 <= hora < 6:
        return 'Madrugada'
    elif 6 <= hora < 12:
        return 'Mañana'
    elif 12 <= hora < 13:
        return 'Mediodia'
    elif 13 <= hora < 18:
        return 'Tarde'
    elif 18 <= hora < 24:
        return 'Noche'
    else:
        return 'Desconocido'
df['franja_horaria'] = df['pickup_hour'].apply(franja_horaria)
df.head()

In [ ]:
# dividir df en train y test
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df.info()

In [ ]:
# mostrar la fila con valores nulos
train_df[train_df.isnull().any(axis=1)]

### Limpieza de los datos
Podemos observar a traves del 'train_df.info' que tenemos 200.000 datos donde las columnas 'dropoff_longitude' y 'dropoff_latitude' poseen un valor faltante en sus datos. Consideramos que al tener una muestra de datos abundante, decidimos eliminarlos del dataset ya que no afecta al entrenamiento del modelo y evitar imputar los datos con un valor promedio. Además, modificamos el tipo de dato de 'pickup_datetime' para verificar que todas las fechas ingresadas hayan sido correctamente cargadas.

Luego de esto, realizamos un '.describe()' sobre las variables numericas y consideramos los resultados para analizar cada variable del dataset.

##### **Análisis de la columna `passenger_count`:**

Rango válido de la variable: [0, 6] pasajeros

A partir del análisis exploratorio mediante `.describe()`, se detectó que el valor máximo registrado en la variable fue de 208 pasajeros por viaje, lo cual constituye un error en la carga de datos. Para visualizar la distribución se realizaron diferentes gráficos:  

- En el **primer boxplot**, se observa claramente el valor atípico de 208, lo que impide una correcta interpretación del resto de los datos.  
- Al excluir dicho valor extremo, el **segundo boxplot** permite observar mejor la dispersión real de la variable.  

Además, se identificaron registros con 0 pasajeros, los cuales también son inconsistentes, ya que un viaje no puede realizarse sin pasajeros.  


In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1, 2, 1)
sns.boxplot(x = train_df['passenger_count'])
plt.title('Boxplot de cantidad de pasajeros sin filtrar')
plt.xlabel('Cantidad de pasajeros')

plt.subplot(1, 2, 2)
sns.boxplot(x = train_df[train_df['passenger_count'] < 208]['passenger_count'])
plt.title('Boxplot de cantidad de pasajero < 208')
plt.xlabel('Cantidad de pasajeros')

plt.show()

In [ ]:
plt.hist(train_df[train_df['passenger_count'] < 208]['passenger_count'], color = 'green')
plt.title('Histograma de cantidad de pasajeros menor a 208')
plt.xlabel('Cantidad de pasajeros')
plt.ylabel('Frecuencia')
plt.show()

Para solucionar este problema y no eliminar dichos registros, se decidió imputar los valores inválidos con la **mediana de la distribución**. La mediana es la medida de tendencia central y es robusta frente a valores atípicos y representa de forma adecuada la cantidad más común de pasajeros en los viajes (generalmente 1). Finalmente, la columna fue convertida a valores enteros para mantener consistencia en la variable.


In [ ]:
from sklearn.impute import SimpleImputer

train_df.loc[train_df['passenger_count'] > 6, 'passenger_count'] = pd.NA
imputer = SimpleImputer(strategy='median')                                      # Imputar con la mediana
train_df['passenger_count'] = imputer.fit_transform(train_df[['passenger_count']])
train_df['passenger_count'] = train_df['passenger_count'].astype(int)

In [ ]:
plt.hist(train_df['passenger_count'], color = 'green')
plt.title('Histograma de cantidad de pasajeros transformado')
plt.xlabel('Cantidad de pasajeros')
plt.ylabel('Frecuencia')
plt.show()

##### Análisis de las columnas `pickup_latitude`, `dropoff_latitude`, `pickup_longitude` y `dropoff_longitude`

Para analizar estos datos, lo primero que realizamos fue la visualización gráfica de cada variable. En estas gráficas se observan bastantes valores atípicos que no corresponden a rangos válidos de latitud y longitud:  

- **Rangos globales válidos**:  
  - Latitud: -90 a 90  
  - Longitud: -180 a 180  

Vamos a considerar todas las coordenadas que caigan dentro de estos rangos o bien tengan una sola coordenada de las 4 posibles fuera del rango, sea cero o nula.
En estos casos imputariamos la coordenada inválida con el algoritmo K-NN.

In [ ]:
plt.figure(figsize=(24,5))

plt.subplot(1, 4, 1)
plt.title("Datos de 'pickup_longitude'")
sns.boxplot(train_df['pickup_longitude'])
plt.ylabel('Longitud de inicio')

plt.subplot(1, 4, 2)
plt.title("Datos de 'dropoff_longitude'")
sns.boxplot(train_df['dropoff_longitude'])
plt.ylabel('Longitud final')

plt.subplot(1, 4, 3)
plt.title("Datos de 'pickup_latitude'")
sns.boxplot(train_df['pickup_latitude'])
plt.ylabel('Latitud de inicio')

plt.subplot(1, 4, 4)
plt.title("Datos de 'dropoff_latitude'")
sns.boxplot(train_df['dropoff_latitude'])
plt.ylabel('Latitud final')

plt.show()

Podemos observar que existen registros en los cuales tanto la latitud como la longitud (de inicio y de destino) tienen valor igual a 0. Estos registros no aportan información relevante al análisis, ya que corresponden a errores de carga, por lo que se decidió **eliminarlos del dataset** para trabajar únicamente con datos reales.  

Por otro lado, aquellos registros en los que solo **una de las coordenadas** (latitud o longitud, ya sea en inicio o destino) tiene valor igual a 0, pueden ser imputados por aproximación. En este caso, en lugar de eliminarlos, se optó por reemplazar dichos valores utilizando el método de imputación por K-vecinos más cercanos (KNN), lo cual permite estimar las coordenadas faltantes en base a los viajes más similares.

In [ ]:
# recorrer el dataset train_df y chequear si las coordenadas son validas o cero o nan y si no los son reemplazarlas con NaN
# si una fila tiene mas de 1 coordenada invalida, eliminar la fila
# mostrar la cantidad de filas eliminadas
# mostrar la cantidad de filas modificadas a NaN

modified_count = 0
initial_rows = train_df.shape[0] # cantidad inicial de filas
for index, row in train_df.iterrows():
    invalid_count = 0

    if not (-90 <= row['pickup_latitude'] <= 90) or abs(row['pickup_latitude']) < 0.1 or pd.isna(row['pickup_latitude']): 
        train_df.at[index, 'pickup_latitude'] = pd.NA
        invalid_count += 1
    if not (-90 <= row['dropoff_latitude'] <= 90 or row['dropoff_latitude'] == 0.0) or pd.isna(row['dropoff_latitude']):
        train_df.at[index, 'dropoff_latitude'] = pd.NA
        invalid_count += 1
    if not (-180 <= row['pickup_longitude'] <= 180) or row['pickup_longitude'] == 0.0 or pd.isna(row['pickup_longitude']):
        train_df.at[index, 'pickup_longitude'] = pd.NA
        invalid_count += 1
    if not (-180 <= row['dropoff_longitude'] <= 180) or row['dropoff_longitude'] == 0.0 or pd.isna(row['dropoff_longitude']):
        train_df.at[index, 'dropoff_longitude'] = pd.NA
        invalid_count += 1
    if invalid_count > 1:
        train_df = train_df.drop(index)
    elif invalid_count == 1:
        modified_count += 1
        # marcar la fila como modificada columna imputada = True
        train_df.at[index, 'imputar'] = True

print(f'Cantidad de filas modificadas a NaN: {modified_count}')
final_rows = train_df.shape[0] # cantidad final de filas
print(f'Cantidad de filas eliminadas: {initial_rows - final_rows}')

In [ ]:
# mostrar las filas imputadas 
train_df[train_df['imputar'] == True]

In [ ]:
from sklearn.impute import KNNImputer

columnas = ['pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']
#k muy pequeño → más sensible a ruido. k muy grande → valores muy suavizados, se pierde variabilidad real.

# Crear imputador KNN
knn_imputer = KNNImputer(n_neighbors=5)
train_df[columnas] = knn_imputer.fit_transform(train_df[columnas])

In [ ]:
#mostrar las filas imputadas 
train_df[train_df['imputar'] == True]

In [ ]:
# falta recalcular la distancia de los registros a imputar, ya que se modificaron las coordenadas



In [ ]:
plt.figure(figsize=(24,5))

plt.subplot(1, 4, 1)
plt.title("Datos de 'pickup_longitude'")
sns.boxplot(train_df['pickup_longitude'], color="skyblue")
plt.ylabel('Longitud de inicio')
plt.ylim(-100, 50)

plt.subplot(1, 4, 2)
plt.title("Datos de 'dropoff_longitude'")
sns.boxplot(train_df['dropoff_longitude'], color="pink")
plt.ylabel('Longitud final')
plt.ylim(-100, 50)

plt.subplot(1, 4, 3)
plt.title("Datos de 'pickup_latitude'")
sns.boxplot(train_df['pickup_latitude'], color="skyblue")
plt.ylabel('Latitud de inicio')
plt.ylim(-100, 60)

plt.subplot(1, 4, 4)
plt.title("Datos de 'dropoff_latitude'")
sns.boxplot(train_df['dropoff_latitude'], color="pink")
plt.ylabel('Latitud final')
plt.ylim(-100, 60)
plt.show()

In [ ]:
# histograma de la distancia en km

plt.hist(train_df['distance_km'], bins=50, color='blue', alpha=0.7)
plt.title('Histograma de la distancia en km')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
# histograma del logaritmo de la distancia en km
plt.hist(np.log1p(train_df['distance_km']), bins=50, color='blue', alpha=0.7)
plt.title('Histograma del logaritmo de la distancia en km')
plt.xlabel('Log(Distancia + 1)')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:
train_df['distance_km'].describe()

In [ ]:
# calcular el primer cuartil (Q1) y el tercer cuartil (Q3)
Q1 = train_df['distance_km'].quantile(0.25).round(2)
Q3 = train_df['distance_km'].quantile(0.75).round(2)
IQR = Q3 - Q1
# definir los límites para los valores atípicos
lower_bound = (Q1 - 5 * IQR).round(2)
upper_bound = (Q3 + 5 * IQR).round(2)
print(f'Q1: {Q1}, Q3: {Q3}, IQR: {IQR}, Lower Bound: {lower_bound}, Upper Bound: {upper_bound}')

In [ ]:
# calcular la cantidad de valores atípicos
outliers = train_df[(train_df['distance_km'] < lower_bound) | (train_df['distance_km'] > upper_bound)]
print(f'Cantidad de valores atípicos: {outliers.shape[0]}')
print(f'Porcentaje de valores atípicos: {outliers.shape[0] / train_df.shape[0] * 100:.2f}%')

In [ ]:
# obtener index de los datos para eliminar los valores atípicos
outlier_indices = outliers.index
# eliminar los datos atípicos de train_df 
train_df = train_df.drop(outlier_indices)


plt.hist(train_df['distance_km'], bins=50, color='blue', alpha=0.7)
plt.title('Histograma de la distancia en km')
plt.xlabel('Distancia (km)')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:
# mostrar cantidad de viajes por año y mes
train_df['pickup_year'] = train_df['pickup_datetime'].dt.year
train_df['pickup_month'] = train_df['pickup_datetime'].dt.month
# mostrar un grafico de barras con la cantidad de viajes por año y mes
plt.figure(figsize=(12,6))
sns.countplot(data=train_df, x='pickup_year', hue='pickup_month', palette='viridis')
plt.title('Cantidad de viajes por año y mes')
plt.xlabel('Año')
plt.ylabel('Cantidad de viajes')
plt.legend(title='Mes', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()  


In [ ]:
# mostrar un scatter plot de la distancia en km vs fare_amount, color franja horaria
plt.figure(figsize=(10,6))
sns.scatterplot(data=train_df, x='distance_km', y='fare_amount', hue='franja_horaria', palette='viridis', alpha=0.6)
plt.title('Distancia en km vs Fare Amount por Franja Horaria')
plt.xlabel('Distancia (km)')
plt.ylabel('Fare Amount')
plt.legend(title='Franja Horaria', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()



tarifas muy altas con poca distancia, habria que revisar...

In [ ]:
# histograma del fare_amount
plt.hist(train_df['fare_amount'], bins=50, color='orange', alpha=0.7)
plt.title('Histograma del fare_amount')
plt.xlabel('Fare Amount')
plt.ylabel('Frecuencia')
plt.show()

In [ ]:
train_df['fare_amount'].describe()

In [ ]:
train_df[train_df['fare_amount'] > 150]

In [ ]:
# mostrar un scatter plot de la distancia en km vs fare_amount, color dia de la semana
plt.figure(figsize=(10,6))
sns.scatterplot(data=train_df, x='distance_km', y='fare_amount', hue='pickup_day_of_week', palette='viridis', alpha=0.6)
plt.title('Distancia en km vs Fare Amount por pickup_day_of_week')
plt.xlabel('Distancia (km)')
plt.ylabel('Fare Amount')
plt.legend(title='pickup_day_of_week', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

In [ ]:
train_df.info()

In [ ]:
# quedaria revisar esas tarifas muy altas con poca distancia ver si las quitamos
# separar train_df en X e y
# aplicar escalado a las variables numericas y codificacion one hot a las categoricas
# definir una metrica para comparar modelos: RMSE, MAE, R2
# aplicar el algoritmo de regresion lineal y evaluar el modelo: descenso del gradiente minibatch